<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/3_rag_texto_completo_semantic_chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruções

* Estratégia 2
    * Usar segmentação por similaridade
    * Busca densa

# Instalando pacotes

In [ ]:
!pip install -q llama-index llama-index-llms-groq sentence-transformers llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Importando pacotes

In [ ]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [ ]:
import json

from llama_index.core import Document
from llama_index.core.response.notebook_utils import display_source_node, display_response
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq
from llama_index.core.evaluation import FaithfulnessEvaluator, CorrectnessEvaluator, RelevancyEvaluator
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser

from IPython.display import Markdown, display
from rich.pretty import pprint
from tqdm import tqdm
import pandas as pd

import nest_asyncio
nest_asyncio.apply()


In [ ]:
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

Cloning into 'ia024-projeto-rag'...
remote: Enumerating objects: 6045, done.
remote: Counting objects: 100% (1050/1050), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 6045 (delta 658), reused 1050 (delta 658), pack-reused 4995
Receiving objects: 100% (6045/6045), 1.12 GiB | 24.18 MiB/s, done.
Resolving deltas: 100% (2047/2047), done.
Updating files: 100% (5787/5787), done.


In [ ]:
DATA_DIR = '/content/ia024-projeto-rag'

# Funções

In [ ]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

In [ ]:
def update_local_repo():
    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

In [ ]:
update_local_repo()

# Criando documentos do llama-index

In [ ]:
from llama_index.core.schema import MetadataMode

docs = []

with open(f"{DATA_DIR}/2-dataset/estatuto.json", 'r') as f:
    estatuto = json.load(f)

metadata={
        "doc_name": estatuto['titulo'],
        "category": "Reitoria"
    }

doc = Document(id_=0
               , text=estatuto['texto_compacto']
               , metadata=metadata
               )

#print(
#    "The LLM sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.LLM),
#)
#print(
#    "The Embedding model sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.EMBED),
#)

docs.append(doc)

In [ ]:
pprint(docs[0])

Document(
│   id_='0',
│   embedding=None,
│   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO I DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º. II - Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. III - Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. IV - Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. V - Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. VI - Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista. VII - Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. VIII - Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. IX - Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. X - Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. XI - Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. XII - Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. XIII - Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. XIV - Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano. XV - Campus Campinas, situado na Rodovia D - Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI - Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva. XVII - Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré. XVIII - Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. XIX - Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. XX - Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. XXI - Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga. XXII - Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro. XXIII - Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. XXIV - Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba. XXV - Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. XXVI - Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. XXVII - Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. XXVIII - Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. XXIX - Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. XXXI - Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila Nova Santana, Assis. § 3º- Para efeito da

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def get_sentences(text):
    sentences = sent_tokenize(text)#, language='portuguese')
    sentences = [s for s in sentences if len(s.strip()) > 2]
    return sentences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sents = get_sentences(estatuto['texto_compacto'])
pprint(sents)

[
│   'ESTATUTO.',
│   'TÍTULO I - DA INSTITUIÇÃO.',
│   'CAPÍTULO I DA NATUREZA E DAS FINALIDADES.',
│   'Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº.',
│   '11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar.',
│   '§1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo.',
│   '§2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei.',
│   'Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º.',
│   'II - Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.',
│   'III - Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.',
│   'IV - Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho.',
│   'V - Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos.',
│   'VI - Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista.',
│   'VII - Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba.',
│   'VIII - Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista.',
│   'IX - Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.',
│   'X - Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque.',
│   'XI - Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos.',
│   'XII - Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão.',
│   'XIII - Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.',
│   'XIV - Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano.',
│   'XV - Campus Campinas, situado na Rodovia D - Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI - Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva.',
│   'XVII - Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré.',
│   'XVIII - Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara.',
│   'XIX - Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga.',
│   'XX - Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui.',
│   'XXI - Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.',
│   'XXII - Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro.',
│   'XXIII - Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio.',
│   'XXIV - Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba.',
│   'XXV - Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia.',
│   'XXVI - Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva.',
│   'XXVII - Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari.',
│   'XXVIII - Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.',
│   'XXIX - Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos.',
│   'XXXI - Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila 

In [ ]:
import spacy
import spacy.cli

spacy.cli.download("pt_core_news_lg")


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load("pt_core_news_lg")
doc = nlp(estatuto['texto_compacto'])
print(doc.text)
for sent in doc.sents:
    print("- ", sent.text)


ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO I DA NATUREZA E DAS FINALIDADES.  Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º. II - Campus São Paulo, situado na Rua P

In [ ]:
from typing import Callable, List

def split_by_sentence_tokenizer() -> Callable[[str], List[str]]:
    import nltk

    tokenizer = nltk.tokenize.PunktSentenceTokenizer()

    # get the spans and then return the sentences
    # using the start index of each span
    # instead of using end, use the start of the next span if available
    def split(text: str) -> List[str]:
        spans = list(tokenizer.span_tokenize(text))
        sentences = []
        for i, span in enumerate(spans):
            start = span[0]
            if i < len(spans) - 1:
                end = spans[i + 1][0]
            else:
                end = len(text)
            sentences.append(text[start:end])

        return sentences

    return split

split_sentences = split_by_sentence_tokenizer()

# Texto a ser dividido em sentenças
texto = estatuto['texto_compacto']

# Usar a função para dividir o texto em sentenças
sentencas = split_sentences(texto)

# Imprimir as sentenças
for sentenca in sentencas:
    print("-",sentenca)

- ESTATUTO. 
- TÍTULO I - DA INSTITUIÇÃO. 
- CAPÍTULO I DA NATUREZA E DAS FINALIDADES.  
- Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 
- 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. 
- §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. 
- §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. 
- Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º. 
- II - Campus S

In [ ]:
import re
# Regex para encontrar sentenças
pattern = r'[^.]+\.(?:\s|$)'

# Encontrar todas as sentenças no texto
sentences = re.findall(pattern, estatuto['texto_compacto'])

pprint(sentences)

[
│   'ESTATUTO. ',
│   'TÍTULO I - DA INSTITUIÇÃO. ',
│   'CAPÍTULO I DA NATUREZA E DAS FINALIDADES. ',
│   ' Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. ',
│   '892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. ',
│   '§1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. ',
│   '§2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. ',
│   'Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º. ',
│   'II - Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. ',
│   'III - Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. ',
│   'IV - Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. ',
│   'V - Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. ',
│   'VI - Campus São João da Boa Vista, situado no Acesso Dr. ',
│   'º, Jardim Itália, São João da Boa Vista. ',
│   'VII - Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. ',
│   'VIII - Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. ',
│   'IX - Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. ',
│   'X - Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. ',
│   'XI - Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. ',
│   'XII - Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. ',
│   'XIII - Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. ',
│   '501, Parque Suzano, Suzano. ',
│   'º, Distrito Industrial, Catanduva. ',
│   'º, Jardim Europa, Avaré. ',
│   'XVIII - Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. ',
│   'XIX - Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. ',
│   'XX - Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. ',
│   'º, Chácara Aviação, Votuporanga. ',
│   '180, Agrochá, Registro. ',
│   'XXIII - Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. ',
│   'º, Bairro Santa Rosa, Piracicaba. ',
│   'XXV - Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. ',
│   'XXVI - Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. ',
│   'XXVII - Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. ',
│   'XXVIII - Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. ',
│   'XXIX - Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. ',
│   'XXXI - Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila Nova Santana, Assis. ',
│   '§ 3º- Para efeito da incidência das disposições que regem a regulação, avaliação e supervisão da Instituição e dos cursos de educação superior, o IFSP é equiparado às universidades federais. ',
│   '§ 4º- O IFSP possui limite de atuação territorial para criar e extinguir cursos, bem como para registrar diplomas dos cursos por ele oferecidos, circunscrito ao Estado de São Paulo, aplicando-se, no caso de oferta de ensino a distância, leg

In [ ]:
docs = []

with open(f"{DATA_DIR}/2-dataset/estatuto.json", 'r') as f:
    estatuto = json.load(f)



for parte in estatuto['partes']:
    metadata = {}
    metadata["doc_name"] = estatuto['titulo']

    titulo_nome    = parte['titulo']
    titulo_artigos = parte['artigos']
    capitulos      = parte['capitulos']

    metadata["titulo"] = titulo_nome
    if titulo_artigos != "":
        doc = Document(text=titulo_artigos, metadata=metadata)
        docs.append(doc)

    for capitulo in capitulos:
        cap_titulo  = capitulo['titulo']
        cap_artigos = capitulo['artigos']

        metadata["capitulo"] = cap_titulo
        doc = Document(text=cap_artigos, metadata=metadata)
        docs.append(doc)

len(docs)
#print(
#    "The LLM sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.LLM),
#)
#print(
#    "The Embedding model sees this: \n",
#    doc.get_content(metadata_mode=MetadataMode.EMBED),
#)

for i, doc in enumerate(docs):
    doc.id_ = f"doc_{i}"

In [ ]:
pprint(docs[0])


Document(
│   id_='doc_0',
│   embedding=None,
│   metadata={
│   │   'doc_name': 'Estatuto',
│   │   'titulo': 'TÍTULO I - DA INSTITUIÇÃO',
│   │   'capitulo': 'CAPÍTULO I DA NATUREZA E DAS FINALIDADES.'
│   },
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º. II - Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. III - Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. IV - Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. V - Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. VI - Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista. VII - Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. VIII - Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. IX - Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. X - Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. XI - Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. XII - Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. XIII - Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. XIV - Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano. XV - Campus Campinas, situado na Rodovia D - Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI - Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva. XVII - Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré. XVIII - Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. XIX - Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. XX - Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. XXI - Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga. XXII - Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro. XXIII - Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. XXIV - Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba. XXV - Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. XXVI - Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. XXVII - Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. XXVIII - Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. XXIX - Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. XXXI - Núcleo Avançado de Assis, situado na Avenida Getúlio Vargas nº 1200, Vila Nova Santana, A

In [ ]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# Esta classe separa o texto em sentenças e, de acordo com o parâmetro "window_size",
# cria janelas com a sentença atual mais 'window_size' sentenças antes e depois
# as janelas são salvas no metadata "window" e as sentenças em "original_sentence"
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=1, # com valor 1, as "janelas" terão tamanho 3: uma sentença antes, a sentença atual, uma sentença depois
    window_metadata_key="window",
    original_text_metadata_key="original_sentence",
)

nodes = node_parser.get_nodes_from_documents(docs)

for i, node in enumerate(nodes):
    node.id_ = f"node_{node.id_}"

len(nodes)



161

In [ ]:
pprint(nodes[:3])

In [ ]:
print(
    "The LLM sees this: \n",
    nodes[1].get_content(metadata_mode=MetadataMode.LLM)
)

The LLM sees this: 
 doc_name: Estatuto
titulo: TÍTULO I - DA INSTITUIÇÃO
capitulo: CAPÍTULO I DA NATUREZA E DAS FINALIDADES.

11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar.


In [ ]:
pprint(nodes[1])

# Fazendo Semantic Chunking

https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="alfaneo/bertimbau-base-portuguese-sts")

splitter = SemanticSplitterNodeParser(
    embed_model=embed_model, breakpoint_percentile_threshold=95, buffer_size=1
)

nodes_semantic = splitter.get_nodes_from_documents(docs, show_progress=True)
for i, node in enumerate(nodes_semantic):
    node.id_ = f"node_{i}"

len(nodes_semantic)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
pprint(nodes_semantic[4])

TextNode(
│   id_='node_4',
│   embedding=None,
│   metadata={
│   │   'doc_name': 'Estatuto',
│   │   'titulo': 'TÍTULO I - DA INSTITUIÇÃO',
│   │   'capitulo': 'CAPÍTULO II DOS PRINCÍPIOS, DAS FINALIDADES, DAS CARACTERÍSTICAS E DOS OBJETIVOS.'
│   },
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   node_id='doc_1',
│   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   metadata={
│   │   │   │   'doc_name': 'Estatuto',
│   │   │   │   'titulo': 'TÍTULO I - DA INSTITUIÇÃO',
│   │   │   │   'capitulo': 'CAPÍTULO II DOS PRINCÍPIOS, DAS FINALIDADES, DAS CARACTERÍSTICAS E DOS OBJETIVOS.'
│   │   │   },
│   │   │   hash='316d34e98a357678aed824c35ebbde2a62509f65ee0342281d1cc7d64b6d485f'
│   │   ),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
│   │   │   node_id='bbc512d3-1cd0-4966-8ba2-e4784be8e1db',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={
│   │   │   │   'doc_name': 'Estatuto',
│   │   │   │   'titulo': 'TÍTULO I - DA INSTITUIÇÃO',
│   │   │   │   'capitulo': 'CAPÍTULO II DOS PRINCÍPIOS, DAS FINALIDADES, DAS CARACTERÍSTICAS E DOS OBJETIVOS.'
│   │   │   },
│   │   │   hash='50d819fa3d429ff1041c54d1a4c93a49e6d47388f5ad29fd621e2fe344f13aec'
│   │   )
│   },
│   text='11.892/2008.',
│   mimetype='text/plain',
│   start_char_idx=4760,
│   end_char_idx=4772,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

In [ ]:
nodes_semantic[4].get_content()

'11.892/2008.'

# Criando e testando VectoreStoreIndex e Metricas

In [ ]:
Settings.embed_model = embed_model
Settings.llm         = Groq(model="llama3-70b-8192")

vector_index = VectorStoreIndex(nodes_semantic)


In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model="unicamp-dl/monoptt5-large", top_n=5)

config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at unicamp-dl/monoptt5-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
node_postprocessors=[
        #MetadataReplacementPostProcessor(target_metadata_key="window"),
        reranker
    ]

query = "Qual o nome do IFSP?"
#query = "Quais são os campi do IFSP? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quantos campi o IFSP tem? Informe onde você encontrou essa informação"
#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"
query_engine = vector_index.as_query_engine(similarity_top_k=10, node_postprocessors=node_postprocessors)
response = query_engine.query(query)
display_response(response)

**`Final Response:`** Os campi do IFSP são:

I - Reitoria, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
II - Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
III - Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.
IV - Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho.
V - Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos.
VI - Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista.
VII - Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba.
VIII - Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista.
IX - Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.
X - Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque.
XI - Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos.
XII - Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão.
XIII - Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.
XIV - Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano.
XV - Campus Campinas, situado na Rodovia D - Pedro I – SP 65 – KM 143,6, Amarais, Campinas;
XVI - Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva.
XVII - Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré.
XVIII - Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara.
XIX - Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga.
XX - Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui.
XXI - Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.
XXII - Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro.
XXIII - Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio.
XXIV - Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba.
XXV - Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia.
XXVI - Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva.
XXVII - Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari.
XXVIII - Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.
XXIX - Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos.

Documento: Estatuto
Capítulo: CAPÍTULO I DA NATUREZA E DAS FINALIDADES
Artigo: Art 1º

In [ ]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

prompts_dict = query_engine.get_prompts()

display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template**Text:** 


Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer such as: Document, Title, Chapter, Article.
Answer in the same language as the query.
Query: {query_str}
Answer: 



**Prompt Key**: response_synthesizer:refine_template**Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [ ]:
from llama_index.core import PromptTemplate
new_summary_tmpl_str = ("""
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer such as: Document, Title, Chapter, Article.
Answer in the same language as the query.
Query: {query_str}
Answer:
"""
)
new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": new_summary_tmpl}
)

In [ ]:
#query = "Qual o nome do IFSP?"
query = "O IFSP tem só ensino médio?" #Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."

response = query_engine.query(query)
display_response(response)

**`Final Response:`** Não, o IFSP não tem só ensino médio.

Source: Document: Estatuto, Title: TÍTULO I - DA INSTITUIÇÃO, Chapter: CAPÍTULO I DA NATUREZA E DAS FINALIDADES, Article: XXXI.

Justification: The article XXXI states that the IFSP is equivalent to federal universities for the purpose of regulating, evaluating, and supervising the institution and higher education courses. This implies that the IFSP offers higher education courses, not just high school education.

In [ ]:
pprint(response)

# Recursive

In [ ]:
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo

base_nodes = [
    TextNode(text=estatuto['texto_compacto'], metadata={"doc_name": estatuto['titulo']})
]
for parte in estatuto['partes']:
    metadata = {}
    metadata["doc_name"] = estatuto['titulo']

    titulo_nome    = parte['titulo']
    titulo_artigos = parte['artigos']
    capitulos      = parte['capitulos']

    metadata["titulo"] = titulo_nome
    if titulo_artigos != "":
        doc = TextNode(text=titulo_artigos, metadata=metadata)
        base_nodes.append(doc)

    for capitulo in capitulos:
        cap_titulo  = capitulo['titulo']
        cap_artigos = capitulo['artigos']

        metadata["capitulo"] = cap_titulo
        doc = TextNode(text=cap_artigos, metadata=metadata)
        base_nodes.append(doc)

len(base_nodes)

18

In [ ]:
pprint(base_nodes[1])

TextNode(
│   id_='a5e9fc4e-1250-483d-b7d7-27656613abb9',
│   embedding=None,
│   metadata={
│   │   'doc_name': 'Estatuto',
│   │   'titulo': 'TÍTULO I - DA INSTITUIÇÃO',
│   │   'capitulo': 'CAPÍTULO I DA NATUREZA E DAS FINALIDADES.'
│   },
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={},
│   text='Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I - Reitoria, no endereço citado no §1º. II - Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo. III - Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão. IV - Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã, Sertãozinho. V - Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de Janeiro, Guarulhos. VI - Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º, Jardim Itália, São João da Boa Vista. VII - Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá, Caraguatatuba. VIII - Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho, 770, Penha, Bragança Paulista. IX - Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto. X - Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição, São Roque. XI - Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119, São Carlos. XII - Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila Abernéssia, Campos do Jordão. XIII - Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos. XIV - Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano, Suzano. XV - Campus Campinas, situado na Rodovia D - Pedro I – SP 65 – KM 143,6, Amarais, Campinas; XVI - Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial, Catanduva. XVII - Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim Europa, Avaré. XVIII - Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza Pinheiro, Araraquara. XIX - Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen, Itapetininga. XX - Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II, Birigui. XXI - Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga. XXII - Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá, Registro. XXIII - Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50, Jardim Tropical, Presidente Epitácio. XXIV - Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro Santa Rosa, Piracicaba. XXV - Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896, Hortolândia. XXVI - Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos Pássaros, Boituva. XXVII - Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João Batista, Capivari. XXVIII - Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão. XXIX - Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145, Jardim Diamante, São José dos Campos. XXXI - Núcleo Avançado de Assis, situado na Avenida Getúlio Varga

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode

# Set Up Recursive Retriever
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [SentenceSplitter(chunk_size=c, chunk_overlap=20) for c in sub_chunk_sizes]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes]
        all_nodes.extend(sub_inodes)
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

all_nodes_dict = {n.node_id: n for n in all_nodes}


vector_index_chunk = VectorStoreIndex(all_nodes, embed_model=embed_model)




In [ ]:
from llama_index.core.retrievers import RecursiveRetriever

vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=15)

retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

# Integrate with RetrieverQueryEngine and Metadata Postprocessing
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from typing import List
from pydantic import BaseModel


class Answer(BaseModel):
    """Data model for an answer."""

    answer: str
    sources: List[str]
    extra_info: str

query_engine_recursive = RetrieverQueryEngine.from_args(
    retriever=retriever_chunk,
    llm=Groq(model="llama3-70b-8192"),
    #output_cls=Answer,
    verbose=True,
    #node_postprocessors=[
    #    MetadataReplacementPostProcessor(target_metadata_key="window")
    #]
)

In [ ]:
pprint(all_nodes[1])

IndexNode(
│   id_='cecc18db-e09d-476a-8463-1308bb8c3ab1',
│   embedding=None,
│   metadata={'doc_name': 'Estatuto'},
│   excluded_embed_metadata_keys=[],
│   excluded_llm_metadata_keys=[],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   node_id='5ef75124-ea6c-4145-bc5e-ca92ab7e339b',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={'doc_name': 'Estatuto'},
│   │   │   hash='fc9b0ec2daff4e1960ffe727f159e7e84381842f6182d9c53a1298f5c96611ed'
│   │   ),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
│   │   │   node_id='110bf141-f4e8-4141-b0c1-d541fa3388ae',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={'doc_name': 'Estatuto'},
│   │   │   hash='89a4c9ed15f2b553dcc761194db60776325f8e346309cbcef197398e5d5d6db7'
│   │   ),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
│   │   │   node_id='35c2c55b-68b9-42a0-b9ef-4d47c66486f0',
│   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   metadata={},
│   │   │   hash='0c64e5a0d561d495754dfffd1e45fc86cd5c78f83fb5a5814a3d6347688f49f4'
│   │   )
│   },
│   text='11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo.',
│   mimetype='text/plain',
│   start_char_idx=233,
│   end_char_idx=544,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n',
│   index_id='5ef75124-ea6c-4145-bc5e-ca92ab7e339b',
│   obj=None
)

In [ ]:
from llama_index.core import PromptTemplate
new_summary_tmpl_str = ("""
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer such as: Document, Title, Chapter, Article.
Answer in the same language as the query.
Query: {query_str}
Source:
Answer:
"""
)

new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)
query_engine_recursive.update_prompts(
    {"response_synthesizer:text_qa_template": new_summary_tmpl}
)

In [ ]:
#query = "Qual o nome do IFSP?"
#query = "Quais são os campi do IFSP? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quantos campi o IFSP tem? Informe onde você encontrou essa informação"
#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
query = "O Art 4º do estatuto do IFSP é sobre o que?"

response = query_engine_recursive.query(query)
display_response(response)

Retrieving with query id None: O Art 4º do estatuto do IFSP é sobre o que?
Retrieved node with id, entering: 5ef75124-ea6c-4145-bc5e-ca92ab7e339b
Retrieving with query id 5ef75124-ea6c-4145-bc5e-ca92ab7e339b: O Art 4º do estatuto do IFSP é sobre o que?
Retrieved node with id, entering: a5e9fc4e-1250-483d-b7d7-27656613abb9
Retrieving with query id a5e9fc4e-1250-483d-b7d7-27656613abb9: O Art 4º do estatuto do IFSP é sobre o que?
Retrieved node with id, entering: 2e6ef48c-fded-4884-80ff-000696ddb229
Retrieving with query id 2e6ef48c-fded-4884-80ff-000696ddb229: O Art 4º do estatuto do IFSP é sobre o que?
Retrieved node with id, entering: b9f16a76-b658-48b1-ad31-b4b79de50594
Retrieving with query id b9f16a76-b658-48b1-ad31-b4b79de50594: O Art 4º do estatuto do IFSP é sobre o que?


**`Final Response:`** Repeat the original answer: O Art 4º do estatuto do IFSP é sobre os princípios norteadores do IFSP.

In [ ]:
display_prompt_dict(query_engine_recursive.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template**Text:** 


Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Provide the source of the answer such as: Document, Title, Chapter, Article.
Answer in the same language as the query.
Query: {query_str}
Answer: 
Source:



**Prompt Key**: response_synthesizer:refine_template**Text:** 

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [ ]:
evaluator = FaithfulnessEvaluator()

eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))
print(str(eval_result.score))

In [ ]:
pprint(eval_result)

In [ ]:
response_str = response.response
for source_node in response.source_nodes:
    eval_result = evaluator.evaluate(
        response=response_str, contexts=[source_node.get_content()]
    )
    print(source_node.id_)
    print(str(eval_result.passing))
    print(str(eval_result.score))

In [ ]:
from llama_index.core.evaluation import RelevancyEvaluator


# define evaluator
evaluator = RelevancyEvaluator()

eval_result = evaluator.evaluate_response(query=query, response=response)
print(str(eval_result.passing))
print(str(eval_result.score))

In [ ]:
response.response

In [ ]:
ref = """
II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.
IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã,
Sertãozinho.
V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de
Janeiro, Guarulhos.
VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º,
Jardim Itália, São João da Boa Vista.
VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá,
Caraguatatuba.
VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho,
770, Penha, Bragança Paulista.
IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.
X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição,
São Roque.
XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119,
São Carlos.
XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila
Abernéssia, Campos do Jordão.
XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.
XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano,
Suzano.
XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais,
Campinas;
XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial,
Catanduva.
XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim
Europa, Avaré.
XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza
Pinheiro, Araraquara.
XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen,
Itapetininga.
XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II,
Birigui.
XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.
XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá,
Registro.
XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50,
Jardim Tropical, Presidente Epitácio.
XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro
Santa Rosa, Piracicaba.
XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896,
Hortolândia.
XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos
Pássaros, Boituva.
XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João
Batista, Capivari.
XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.
XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145,
Jardim Diamante, São José dos Campos.
"""

from llama_index.core.evaluation import CorrectnessEvaluator

evaluator = CorrectnessEvaluator()

eval_result = evaluator.evaluate(
    query=query,
    response=response.response,
    reference=ref,
)

print(str(eval_result.score))
print(str(eval_result.feedback))


4.0
The generated answer is relevant to the user query and lists most of the campuses of IFSP, but it is not fully correct as it is missing one campus (São Paulo) and has an incorrect entry (XXVIII - Núcleo Avançado de Assis, which is not a campus).


# Avaliando conjunto de perguntas e respostas

In [ ]:
pairs = [{
            "question": "Qual o nome do IFSP?",
            "answer": "Instituto Federal de Educação, Ciência e Tecnologia de São Paulo.  Referência: Estatuto, Título I, Capítulo I, Art. 1º"
         }
         ,{
            "question": "Quais são os campi do IFSP?",
            "answer": """II. Campus São Paulo, situado na Rua Pedro Vicente, 625, Canindé, São Paulo.
III. Campus Cubatão, situado na Rua Maria Cristina, 50, Jardim Casqueiro, Cubatão.
IV. Campus Sertãozinho, situado na Rua Américo Ambrósio, 269, Jardim Canaã,
Sertãozinho.
V. Campus Guarulhos, situado na Avenida Salgado Filho, 2501, Bairro Vila Rio de
Janeiro, Guarulhos.
VI. Campus São João da Boa Vista, situado no Acesso Dr. João Batista Merlin, s/ n.º,
Jardim Itália, São João da Boa Vista.
VII. Campus Caraguatatuba, situado na Avenida Rio Grande do Norte, 450, Indaiá,
Caraguatatuba.
VIII. Campus Bragança Paulista, situado na Avenida Francisco Samuel Lucchesi Filho,
770, Penha, Bragança Paulista.
IX. Campus Salto, situado na Rua Rio Branco, 1780, Vila Teixeira, Salto.
X. Campus São Roque, situado na Rodovia Quintino de Lima, 2100, Jardim Conceição,
São Roque.
XI. Campus São Carlos, situado na Rodovia Washington Luís, km 235 AT-6, Sala 119,
São Carlos.
XII. Campus Campos do Jordão, situado na Rua Monsenhor José Vita, 280, Vila
Abernéssia, Campos do Jordão.
XIII. Campus Barretos, situado na Avenida C-1, Bairro Ide Daher, Barretos.
XIV. Campus Suzano, situado na Avenida Mogi das Cruzes, 1.501, Parque Suzano,
Suzano.
XV. Campus Campinas, situado na Rodovia D. Pedro I – SP 65 – KM 143,6, Amarais,
Campinas;
XVI. Campus Catanduva, situado na Avenida Imperatriz, s/ n.º, Distrito Industrial,
Catanduva.
XVII. Campus Avaré, situado na Avenida Professor Celso Ferreira da Silva, s/ n.º, Jardim
Europa, Avaré.
XVIII. Campus Araraquara, situado no Ramal de Acesso Engenheiro Heitor de Souza
Pinheiro, Araraquara.
XIX. Campus Itapetininga, situado na Avenida João Olímpio de Oliveira, Bairro Assen,
Itapetininga.
XX. Campus Birigui, situado na Rua Pedro Cavalo, 709, Residencial Portal da Pérola II,
Birigui.
XXI. Campus Votuporanga, situado na Rua Pará, s/ n.º, Chácara Aviação, Votuporanga.
XXII. Campus Registro, situado na Rua Clara Gianotti de Souza, 5.180, Agrochá,
Registro.
XXIII. Campus Presidente Epitácio, situado na Rua Rua José Ramos Júnior, nº 27-50,
Jardim Tropical, Presidente Epitácio.
XXIV. Campus Piracicaba, situado na Rodovia Deputado Laércio Corte, s/ n.º, Bairro
Santa Rosa, Piracicaba.
XXV. Campus Hortolândia, situado na Rodovia Municipal TeodorCondiev, 1896,
Hortolândia.
XXVI. Campus Boituva, situado na Avenida Zélia de Lima Rosa, nº 100, Portal dos
Pássaros, Boituva.
XXVII. Campus Capivari, situado na Avenida Ênio Pires de Camargo, 2971, São João
Batista, Capivari.
XXVIII. Campus Matão, situado na Rua José Bonifácio, 1176, Centro, Matão.
XXIX. Campus São José dos Campos, situado na Rodovia Presidente Dutra s/n – km 145,
Jardim Diamante, São José dos Campos.

Referência: Estatuto, Título I, Capítulo I, Art. 1º, §2º
"""
         }
         ,{
            "question": "Quantos campi o IFSP tem? Informe onde você encontrou essa informação",
            "answer": "O IFSP tem 28 campi. Referência: Estatuto, Capítulo I, Art. 1º, §2º"
         }
        ]

#query = "O IFSP tem só ensino médio? Além de responder à pergunta, informe o documento, o Capítulo e o Artigo de referência."
#query = "Quais artigos do Estatuto dizem que o IFSP oferece mestrado e doutorado?"
#query = "O que diz o Art 6º do estatuto do IFSP?"
#query = "O Art 4º do estatuto do IFSP é sobre o que?"

In [ ]:
faith_evaluator     = FaithfulnessEvaluator()
relevancy_evaluator = RelevancyEvaluator()
correct_evaluator   = CorrectnessEvaluator()
data = []
instruction = "\n\nInforme os dados: Documento, Título, Capítulo e Artigo que embasam a resposta."
for pair in tqdm(pairs):
    result       = {}
    query        = pair['question']
    ground_truth = pair['answer']

    result['question']     = query
    result['ground_truth'] = ground_truth

    # Fazendo consulta
    query_engine = vector_index.as_query_engine(similarity_top_k=10)
    response = query_engine.query(query + instruction)
    result['answer'] = response.response

    # Avaliando Faithfulness
    eval_result = faith_evaluator.evaluate_response(response=response)
    result['faithfulness'] = eval_result.score

    # Avaliando Relevancy
    eval_result = relevancy_evaluator.evaluate_response(query=query, response=response)
    result['relevancy'] = eval_result.score

    # Avaliando Correctness
    eval_result = correct_evaluator.evaluate(
        query=query,
        response=response.response,
        reference=ground_truth,
    )
    result['correctness'] = eval_result.score
    result['correcteness_feedback'] = eval_result.feedback

    data.append(result)


df = pd.DataFrame(data)
df

100%|██████████| 3/3 [00:46<00:00, 15.38s/it]


,question,ground_truth,answer,faithfulness,relevancy,correctness,correcteness_feedback
0,Qual o nome do IFSP?,"Instituto Federal de Educação, Ciência e Tecno...",O nome do IFSP é o Instituto Federal de Educaç...,1.0,1.0,5.0,The generated answer is fully correct and rele...
1,Quais são os campi do IFSP?,"II. Campus São Paulo, situado na Rua Pedro Vic...","Os campi do IFSP são: Campus Matão, Campus São...",1.0,1.0,2.0,The generated answer is relevant to the user q...
2,Quantos campi o IFSP tem? Informe onde você en...,"O IFSP tem 28 campi. Referência: Estatuto, Cap...",The IFSP has 3 campuses.\n\nI found this infor...,1.0,1.0,2.0,The generated answer is relevant to the user q...


In [ ]:
# Definindo as tools disponíveis
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="search",
            description=(
                "Provides context to answer the questions. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

agent = ReActAgent.from_tools(
    query_engine_tools,
    #verbose=True # mostra as mensagens na saída
)


faith_evaluator     = FaithfulnessEvaluator()
relevancy_evaluator = RelevancyEvaluator()
correct_evaluator   = CorrectnessEvaluator()
data = []
instruction = "\n\nInforme os dados: Documento, Título, Capítulo e Artigo que embasam a resposta."
for pair in tqdm(pairs):
    result       = {}
    query        = pair['question']
    ground_truth = pair['answer']

    result['question']     = query
    result['ground_truth'] = ground_truth

    # Fazendo consulta
    # Apaga o histórico do agent para executar a próxima consulta
    agent.reset()

    response = agent.chat(query + instruction)

    result['answer'] = response.response

    # Avaliando Faithfulness
    eval_result = faith_evaluator.evaluate_response(response=response)
    result['faithfulness'] = eval_result.score

    # Avaliando Relevancy
    eval_result = relevancy_evaluator.evaluate_response(query=query, response=response)
    result['relevancy'] = eval_result.score

    # Avaliando Correctness
    eval_result = correct_evaluator.evaluate(
        query=query,
        response=response.response,
        reference=ground_truth,
    )
    result['correctness'] = eval_result.score
    result['correcteness_feedback'] = eval_result.feedback

    data.append(result)


df = pd.DataFrame(data)
df

100%|██████████| 3/3 [03:17<00:00, 65.78s/it]


,question,ground_truth,answer,faithfulness,relevancy,correctness,correcteness_feedback
0,Qual o nome do IFSP?,"Instituto Federal de Educação, Ciência e Tecno...",O nome do IFSP é Instituto Federal de São Paulo.,1.0,1.0,4.5,The generated answer is very close to the refe...
1,Quais são os campi do IFSP?,"II. Campus São Paulo, situado na Rua Pedro Vic...","Os campi do IFSP são: Campus Matão, Campus São...",1.0,1.0,4.0,The generated answer is relevant to the user q...
2,Quantos campi o IFSP tem? Informe onde você en...,"O IFSP tem 28 campi. Referência: Estatuto, Cap...","O IFSP tem 3 campi: Campus Matão, Campus São J...",1.0,1.0,2.0,The generated answer is relevant to the user q...


In [ ]:
pprint(response)

AgentChatResponse(
│   response='O IFSP tem 3 campi: Campus Matão, Campus São José dos Campos e Núcleo Avançado de Assis.',
│   sources=[
│   │   ToolOutput(
│   │   │   content='2',
│   │   │   tool_name='search',
│   │   │   raw_input={'input': 'Quantos campi o IFSP tem?'},
│   │   │   raw_output=Response(
│   │   │   │   response='2',
│   │   │   │   source_nodes=[
│   │   │   │   │   NodeWithScore(
│   │   │   │   │   │   node=TextNode(
│   │   │   │   │   │   │   id_='node_8',
│   │   │   │   │   │   │   embedding=None,
│   │   │   │   │   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   │   │   │   │   excluded_embed_metadata_keys=[],
│   │   │   │   │   │   │   excluded_llm_metadata_keys=[],
│   │   │   │   │   │   │   relationships={
│   │   │   │   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   │   │   │   │   │   │   node_id='0',
│   │   │   │   │   │   │   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   │   │   │   │   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   │   │   │   │   │   │   hash='512339ef4716e689fd625829f2898076750e246a255a52e28549524925cd7f46'
│   │   │   │   │   │   │   │   ),
│   │   │   │   │   │   │   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
│   │   │   │   │   │   │   │   │   node_id='db2b43e6-1a5d-48a0-96eb-1ebdf7d117be',
│   │   │   │   │   │   │   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   │   │   │   │   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   │   │   │   │   │   │   hash='6c05a30464a0a4add3ad70757dd71f9925af5e86e78e8a79febf095f0972df25'
│   │   │   │   │   │   │   │   ),
│   │   │   │   │   │   │   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
│   │   │   │   │   │   │   │   │   node_id='0c0ef6e6-fbb0-4f8e-ac36-96d3c1c913d8',
│   │   │   │   │   │   │   │   │   node_type=<ObjectType.TEXT: '1'>,
│   │   │   │   │   │   │   │   │   metadata={},
│   │   │   │   │   │   │   │   │   hash='4b655d1e86f60c267638b0170c1d111f7032262c6d49901c44af11ea603e2afc'
│   │   │   │   │   │   │   │   )
│   │   │   │   │   │   │   },
│   │   │   │   │   │   │   text='§ 3º- Para efeito da incidência das disposições que regem a regulação, avaliação e supervisão da Instituição e dos cursos de educação superior, o IFSP é equiparado às universidades federais. § 4º- O IFSP possui limite de atuação territorial para criar e extinguir cursos, bem como para registrar diplomas dos cursos por ele oferecidos, circunscrito ao Estado de São Paulo, aplicando-se, no caso de oferta de ensino a distância, legislação específica. Art 2º - O IFSP rege-se pelos atos normativos mencionados no caput do Art 1º -, pela legislação federal e pelos seguintes instrumentos normativos: I - Estatuto. ',
│   │   │   │   │   │   │   mimetype='text/plain',
│   │   │   │   │   │   │   start_char_idx=3770,
│   │   │   │   │   │   │   end_char_idx=4382,
│   │   │   │   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   │   │   │   metadata_seperator='\n'
│   │   │   │   │   │   ),
│   │   │   │   │   │   score=0.7826719376241549
│   │   │   │   │   ),
│   │   │   │   │   NodeWithScore(
│   │   │   │   │   │   node=TextNode(
│   │   │   │   │   │   │   id_='node_42',
│   │   │   │   │   │   │   embedding=None,
│   │   │   │   │   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   │   │   │   │   excluded_embed_metadata_keys=[],
│   │   │   │   │   │   │   excluded_llm_metadata_keys=[],
│   │   │   │   │   │   │   relationships={
│   │   │   │   │   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   │   │   │   │   │   │   node_id='0',
│   │   │   │   │   │   │   │   │   node_type=<ObjectType.DOCUMENT: '4'>,
│   │   │   │   │   │   │   │   │   metadata={'doc_name': 'Estatuto', 'category': 'Reitoria'},
│   │   │   │   │   │   │   │   │   hash='512339ef4